In [3]:
import os
import shutil
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.ndimage import binary_fill_holes
from skimage.measure import label, regionprops
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img, load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score, classification_report
import json
import random
import h5py  # h5py is used for handling HDF5 files
from tensorflow.keras.models import model_from_json  # This imports the function to load Keras models

BACKGROUND REMOVAL

In [ ]:
# BACKGROUND REMOVAL

# Define the input and output folder paths
# `input_folder` contains the raw images to be processed.
# `output_folder` is where the processed images will be saved.
input_folder = '/dataset tfg final'
output_folder = '/preprocess final'

# Path to store preprocessed images, organized by grade
preprocessed_folder = os.path.join(output_folder, 'preprocessed')

# Function to determine the grade of an image based on its filename
def determine_grade(filename):
    """
    Determines the grade of osteoarthritis based on the filename.

    Args:
        filename (str): The name of the image file.

    Returns:
        int: The grade of osteoarthritis (0, 1, 2, or 3), or None if not identifiable.
    """
    if "Gr0" in filename:
        return 0
    elif "Gr1" in filename:
        return 1
    elif "Gr2" in filename:
        return 2
    elif "Gr3" in filename:
        return 3
    else:
        return None

# Function to process an individual image
def process_image(input_path, output_path, filename):
    """
    Processes a single image: removes the background, applies segmentation,
    and saves the preprocessed image into a corresponding folder.

    Args:
        input_path (str): The folder path where the image is located.
        output_path (str): The folder path where the processed image will be saved.
        filename (str): The name of the image file.
    """
    # Full paths for the input and output image
    input_file_path = os.path.join(input_path, filename)
    output_file_path = os.path.join(output_path, filename)

    try:
        # Process only specific images (e.g., "originales" and "saf" in filename)
        if "originales" in input_path.lower() and ("saf" in filename.lower() or "safo" in filename.lower()):
            # Load the image in color
            color_image = cv2.imread(input_file_path)

            # Crop the image to a fixed size (e.g., 3072 pixels height)
            cropped_image = color_image[:3072, :]

            # Convert the image to grayscale
            gray_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)

            # Identify black pixels in the grayscale image
            black_pixels = (gray_image == 0)

            # Create a mask for black pixels
            black_pixels_mask = np.zeros_like(gray_image)
            black_pixels_mask[black_pixels] = 255

            # Dilate the black pixel mask to expand the regions
            kernel = np.ones((300, 300), np.uint8)
            dilated_mask = cv2.dilate(black_pixels_mask, kernel, iterations=1)

            # Identify adjacent white pixels (potential tissue borders)
            adjacent_white_pixels = (cv2.dilate(dilated_mask, np.ones((9, 9), np.uint8), iterations=1) - dilated_mask) > 0

            # Compute the average intensity of the adjacent white pixels
            whitish_tone = np.mean(gray_image[adjacent_white_pixels])

            # Handle cases where the computed tone is not finite
            if not np.isfinite(whitish_tone):
                whitish_tone = 220  # Default value for missing intensity

            # Replace the dilated black regions with the whitish tone
            gray_image[dilated_mask > 0] = whitish_tone

            # Apply Otsu's thresholding to binarize the grayscale image
            _, otsu_threshold = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

            # Invert the binary mask to focus on black regions
            inverted_otsu_threshold = cv2.bitwise_not(otsu_threshold)

            # Fill holes in the binary mask
            filled_image = binary_fill_holes(inverted_otsu_threshold).astype(np.uint8) * 255

            # Label connected components in the binary mask
            labeled_image, num_features = label(filled_image, return_num=True, connectivity=2)

            # Get properties of the labeled regions
            regions = regionprops(labeled_image)

            # Identify the largest connected region by area
            if regions:
                largest_region = max(regions, key=lambda r: r.area)
                largest_region_mask = (labeled_image == largest_region.label).astype(np.uint8) * 255
            else:
                largest_region_mask = filled_image  # Default to the filled image if no regions are found

            # Apply the mask to retain only the largest region in the color image
            final_color_image = cv2.bitwise_and(cropped_image, cropped_image, mask=largest_region_mask)

            # Visualize intermediate steps (optional, for debugging or demonstration)
            plt.figure(figsize=(18, 6))
            plt.subplot(1, 4, 1)
            plt.imshow(cv2.cvtColor(otsu_threshold, cv2.COLOR_BGR2RGB))
            plt.axis('off')
            plt.title("Otsu Threshold")

            plt.subplot(1, 4, 2)
            plt.imshow(filled_image, cmap='gray')
            plt.axis('off')
            plt.title("Filled Holes")

            plt.subplot(1, 4, 3)
            plt.imshow(largest_region_mask, cmap='gray')
            plt.axis('off')
            plt.title("Largest Region Mask")

            plt.subplot(1, 4, 4)
            plt.imshow(cv2.cvtColor(final_color_image, cv2.COLOR_BGR2RGB))
            plt.axis('off')
            plt.title("Final Color Image")

            plt.tight_layout()
            plt.show()  # Display the processed images

            # Determine the grade of the image using the filename
            grade = determine_grade(filename)
            if grade is not None:
                # Create a folder for the specific grade
                grade_folder = os.path.join(preprocessed_folder, f'grade{grade}')
                os.makedirs(grade_folder, exist_ok=True)

                # Save the processed image with a new name
                output_filename = f"processed_{filename}"
                preprocessed_output_path = os.path.join(grade_folder, output_filename)
                cv2.imwrite(preprocessed_output_path, final_color_image)  # Save the processed image

    except Exception as e:
        # Handle any errors that occur during processing
        print(f"Failed to process the image: {input_file_path}")
        print(f"Error: {str(e)}")

# Remove the output folder if it already exists and create a clean one
if os.path.exists(output_folder):
    shutil.rmtree(output_folder)  # Delete the folder and its contents
os.makedirs(output_folder)
os.makedirs(preprocessed_folder)

# Walk through the input folder to find all image files
for root, folders, files in os.walk(input_folder):
    for filename in files:
        # Process only specific image formats
        if filename.endswith(('.jpg', '.jpeg', '.tif', '.tiff')):
            process_image(root, output_folder, filename)  # Process each valid image

print('Process completed.')  # Indicate that the process has finished



BALANCE DATASET AND TRAIN/TEST SEPARATION

In [ ]:
# BALANCE DATASET AND TRAIN/TEST SEPARATION

# Define the input and output folder paths
# `input_folder` contains preprocessed images.
# `output_folder` is where the balanced train and test sets will be stored.
input_folder = '/preprocess final'
output_folder = '/safo balanced final'

# Define subfolders for train and test splits
train_folder = os.path.join(output_folder, 'train')
test_folder = os.path.join(output_folder, 'test')

# Function to determine the grade of an image based on its filename
def determine_grade(filename):
    """
    Determines the grade of osteoarthritis based on the filename.

    Args:
        filename (str): Name of the image file.

    Returns:
        int: The grade of osteoarthritis (0, 1, 2, or 3), or None if not identifiable.
    """
    if "Gr0" in filename:
        return 0
    elif "Gr1" in filename:
        return 1
    elif "Gr2" in filename:
        return 2
    elif "Gr3" in filename:
        return 3
    else:
        return None

# Create the output folders for train and test sets
# Ensures that the required folders exist for each grade in train and test splits.
os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

# Create subfolders for each grade (grade0, grade1, grade2, grade3)
for grade in range(4):
    os.makedirs(os.path.join(train_folder, f'grade{grade}'), exist_ok=True)
    os.makedirs(os.path.join(test_folder, f'grade{grade}'), exist_ok=True)

# Traverse all images in the input folder
# Collect all image file paths from the input folder.
image_files = []
for root, folders, files in os.walk(input_folder):
    for filename in files:
        # Only include specific image file formats
        if filename.endswith(('.jpg', '.jpeg', '.tif', '.tiff')):
            image_files.append(os.path.join(root, filename))

# Dictionary to store images grouped by grade
# Keys are grades (0, 1, 2, 3), values are lists of image paths.
images_by_grade = {0: [], 1: [], 2: [], 3: []}

# Categorize images into grades based on their filenames
for image_path in image_files:
    filename = os.path.basename(image_path)  # Extract just the filename from the full path
    grade = determine_grade(filename)  # Determine the grade using the filename
    if grade is not None:
        images_by_grade[grade].append(image_path)  # Add the image to the corresponding grade list

# Balance and split the dataset into train and test sets for each grade
for grade, images in images_by_grade.items():
    random.shuffle(images)  # Shuffle the images randomly to ensure a balanced split

    # Define train and test splits for each grade based on specific rules
    if grade == 0:  # Grade 0: 5 images for train, 1 for test
        train_images = images[:5]
        test_images = images[5:6]
    elif grade == 1:  # Grade 1: 7 images for train, rest for test
        train_images = images[:7]
        test_images = images[7:]
    elif grade == 2:  # Grade 2: 7 images for train, rest for test
        train_images = images[:7]
        test_images = images[7:]
    elif grade == 3:  # Grade 3: 4 images for train, 1 for test
        train_images = images[:4]
        test_images = images[4:5]

    # Copy the images to their respective train and test folders
    for img_path in train_images:
        shutil.copy(img_path, os.path.join(train_folder, f'grade{grade}'))  # Copy to train folder
    for img_path in test_images:
        shutil.copy(img_path, os.path.join(test_folder, f'grade{grade}'))  # Copy to test folder

# Print a confirmation message once the process is complete
print('Dataset balanced and split into train and test sets.')



PATCHES

In [ ]:
# IMAGE PATCH CREATION

# Define the input and output folder paths for image patches
# `patches_train_folder` is the folder where training image patches will be saved.
# `patches_test_folder` is the folder where testing image patches will be saved.
patches_train_folder = '/safo balanced final/trainpatches'
patches_test_folder = '/safo balanced final/testpatches'

# Create the output folders for storing patches
# Ensures that the required folders exist for patches for both training and testing sets.
os.makedirs(patches_train_folder, exist_ok=True)
os.makedirs(patches_test_folder, exist_ok=True)

# Create subfolders for each grade (grade0, grade1, grade2, grade3)
for grade in range(4):
    os.makedirs(os.path.join(patches_train_folder, f'grade{grade}'), exist_ok=True)
    os.makedirs(os.path.join(patches_test_folder, f'grade{grade}'), exist_ok=True)

# Function to create patches from an image
def create_patches(image_path, output_folder, patch_size=(3072, 3072), stride=512):
    """
    Splits an image into smaller patches and saves them to the specified folder.

    Args:
        image_path (str): The path to the image file.
        output_folder (str): The folder where the patches will be saved.
        patch_size (tuple): The size of each patch (height, width).
        stride (int): The step size for moving the patch window.
    """
    # Read the input image
    image = cv2.imread(image_path)

    # Get the dimensions of the image
    image_height, image_width, _ = image.shape
    patch_count = 0  # Counter for naming the patches

    # Iterate over the image using the defined stride
    for y in range(0, image_height, stride):
        for x in range(0, image_width, stride):
            # Ensure the patch stays within the image boundaries
            if y + patch_size[1] <= image_height and x + patch_size[0] <= image_width:
                # Extract the patch
                patch = image[y:y + patch_size[1], x:x + patch_size[0]]

                # Define the filename for the patch
                patch_filename = f'{os.path.splitext(os.path.basename(image_path))[0]}_patch_{patch_count}.png'

                # Save the patch to the output folder
                patch_output_path = os.path.join(output_folder, patch_filename)
                cv2.imwrite(patch_output_path, patch)

                # Increment the patch counter
                patch_count += 1

# Create patches for training images
# Iterates through each grade folder in the training set and creates patches for each image.
for grade in range(4):
    train_grade_folder = os.path.join(train_folder, f'grade{grade}')
    patches_train_grade_folder = os.path.join(patches_train_folder, f'grade{grade}')

    # Process each image in the current grade folder
    for filename in os.listdir(train_grade_folder):
        if filename.endswith(('.jpg', '.jpeg', '.tif', '.tiff', '.png')):  # Filter supported image formats
            img_path = os.path.join(train_grade_folder, filename)
            create_patches(img_path, patches_train_grade_folder)  # Create patches for the image

# Create patches for testing images
# Similar to the training set, but operates on the test set.
for grade in range(4):
    test_grade_folder = os.path.join(test_folder, f'grade{grade}')
    patches_test_grade_folder = os.path.join(patches_test_folder, f'grade{grade}')

    # Process each image in the current grade folder
    for filename in os.listdir(test_grade_folder):
        if filename.endswith(('.jpg', '.jpeg', '.tif', '.tiff', '.png')):  # Filter supported image formats
            img_path = os.path.join(test_grade_folder, filename)
            create_patches(img_path, patches_test_grade_folder)  # Create patches for the image

# Print a confirmation message
print('All images have been patched.')


REMOVE PATCHES WITH LESS THAN 30% OF TISSUE


In [ ]:
# REMOVE PATCHES WITH LESS THAN 30% TISSUE

# Define the input folder paths for training and testing image patches
# `patches_train_folder` contains the patches from the training set.
# `patches_test_folder` contains the patches from the testing set.
patches_train_folder = '/safo balanced final/trainpatches'
patches_test_folder = '/safo balanced final/testpatches'

# Define the minimum percentage of tissue (non-black pixels) required to keep a patch
min_tissue_percentage = 0.30  # 30% of the patch must contain tissue

# Function to calculate the percentage of tissue in a patch
def calculate_tissue_percentage(patch):
    """
    Calculates the percentage of tissue (non-black pixels) in an image patch.

    Args:
        patch (numpy array): The input image patch.

    Returns:
        float: The percentage of the patch containing tissue.
    """
    # Convert the patch to grayscale for easier processing
    gray_patch = cv2.cvtColor(patch, cv2.COLOR_BGR2GRAY)

    # Apply a binary threshold to identify non-black pixels (tissue)
    _, thresholded_patch = cv2.threshold(gray_patch, 1, 255, cv2.THRESH_BINARY)

    # Count the number of tissue pixels (non-black)
    tissue_pixels = np.sum(thresholded_patch == 255)

    # Calculate the total number of pixels in the patch
    total_pixels = thresholded_patch.size

    # Compute the tissue percentage
    tissue_percentage = tissue_pixels / total_pixels

    return tissue_percentage

# Function to remove patches with insufficient tissue
def remove_patches_with_little_tissue(patch_folder):
    """
    Removes patches from a folder if their tissue percentage is below the defined threshold.

    Args:
        patch_folder (str): The folder containing patches organized by grades.
    """
    for grade in range(4):  # Iterate through each grade folder (grade0, grade1, grade2, grade3)
        grade_folder = os.path.join(patch_folder, f'grade{grade}')

        for filename in os.listdir(grade_folder):
            if filename.endswith(('.jpg', '.jpeg', '.tif', '.tiff', '.png')):  # Filter image files
                patch_path = os.path.join(grade_folder, filename)

                # Read the patch image
                patch = cv2.imread(patch_path)

                # Calculate the percentage of tissue in the patch
                tissue_percentage = calculate_tissue_percentage(patch)

                # Remove the patch if the tissue percentage is below the threshold
                if tissue_percentage < min_tissue_percentage:
                    os.remove(patch_path)  # Delete the patch
                    print(f'Removed patch: {patch_path} (tissue percentage: {tissue_percentage:.2f})')

# Remove patches with insufficient tissue from both training and testing sets
remove_patches_with_little_tissue(patches_train_folder)
remove_patches_with_little_tissue(patches_test_folder)

# Print a confirmation message once the process is complete
print('Patches with insufficient tissue have been removed.')


PATCHES BALANCE

In [ ]:
# Directories for the training and testing patches
# `train_dir` contains the patches for training.
# `test_dir` contains the patches for testing.
train_dir = '/safo balanced final/trainpatches'
test_dir = '/safo balanced final/testpatches'

# Desired percentages for each grade
# These percentages define how much of each grade's images will be moved from training to testing.
percentages = {
    'grade0': 0.20,  # Move 20% of Grade 0 patches to the test set
    'grade1': 0.30,  # Move 30% of Grade 1 patches to the test set
    'grade2': 0.30,  # Move 30% of Grade 2 patches to the test set
    'grade3': 0.20   # Move 20% of Grade 3 patches to the test set
}

# Function to move images from one folder to another
def move_images(src_folder, dest_folder, percentage):
    """
    Moves a percentage of images from the source folder to the destination folder.

    Args:
        src_folder (str): Path to the source folder containing the images.
        dest_folder (str): Path to the destination folder where images will be moved.
        percentage (float): Percentage of images to move (value between 0 and 1).
    """
    # Get the list of image filenames in the source folder
    images = os.listdir(src_folder)

    # Calculate the number of images to move based on the percentage
    num_images_to_move = int(len(images) * percentage)

    # Randomly select the images to move
    images_to_move = random.sample(images, num_images_to_move)

    # Move the selected images to the destination folder
    for image in images_to_move:
        src_path = os.path.join(src_folder, image)  # Full path to the source image
        dest_path = os.path.join(dest_folder, image)  # Full path to the destination
        shutil.move(src_path, dest_path)  # Move the image

    # Print a summary of the operation
    print(f"{num_images_to_move} images moved from {src_folder} to {dest_folder}")

# Move images for each grade
# Iterate over the grade groups and their corresponding percentages
for group, percentage in percentages.items():
    group_train_dir = os.path.join(train_dir, group)  # Training folder for the current grade
    group_test_dir = os.path.join(test_dir, group)  # Testing folder for the current grade

    # Ensure the destination folder exists
    if not os.path.exists(group_test_dir):
        os.makedirs(group_test_dir)  # Create the folder if it doesn't exist

    # Move the images for the current grade based on the specified percentage
    move_images(group_train_dir, group_test_dir, percentage)


RESIZED IMAGES

In [ ]:
# RESIZE IMAGE PATCHES TO 512x512

# Define the input and output folder paths for the patches
# `patches_train_folder` contains the patches for the training set.
# `patches_test_folder` contains the patches for the testing set.
patches_train_folder = '/safo balanced final/trainpatches'
patches_test_folder = '/safo balanced final/testpatches'

# Define the output folder paths for the resized patches
# `resized_train_folder` will store the resized patches for training.
# `resized_test_folder` will store the resized patches for testing.
resized_train_folder = '/safo balanced final/resizedtrainpatches'
resized_test_folder = '/safo balanced final/resizedtestpatches'

# Define the output size for the resized images (512x512)
output_size = (512, 512)

# Create the output folders if they don't already exist
# Ensures that the required folders exist for resized training and testing patches.
os.makedirs(resized_train_folder, exist_ok=True)
os.makedirs(resized_test_folder, exist_ok=True)

# Function to resize an image to the desired size
def resize_image(image, size=(512, 512)):
    """
    Resizes the input image to the specified size.

    Args:
        image (numpy array): The input image to be resized.
        size (tuple): The target size (width, height) for the resized image.

    Returns:
        numpy array: The resized image.
    """
    # Use INTER_AREA interpolation method for resampling (better for shrinking images)
    resized_image = cv2.resize(image, size, interpolation=cv2.INTER_AREA)
    return resized_image

# Function to resize all patches in a folder
def resize_patches_in_folder(patch_folder, resized_folder):
    """
    Iterates through all patches in the given folder, resizes them, and saves them to the resized folder.

    Args:
        patch_folder (str): Path to the folder containing image patches to be resized.
        resized_folder (str): Path to the folder where the resized patches will be saved.
    """
    # Iterate through each grade (0 to 3) and resize patches for each grade folder
    for grade in range(4):
        grade_folder = os.path.join(patch_folder, f'grade{grade}')
        resized_grade_folder = os.path.join(resized_folder, f'grade{grade}')

        # Create the subfolder for each grade if it does not exist
        os.makedirs(resized_grade_folder, exist_ok=True)

        # Iterate through all files in the current grade folder
        for filename in os.listdir(grade_folder):
            if filename.endswith(('.jpg', '.jpeg', '.tif', '.tiff', '.png')):  # Only process image files
                patch_path = os.path.join(grade_folder, filename)

                # Read the patch image
                patch = cv2.imread(patch_path)

                if patch is not None:
                    # Resize the image using the resize_image function
                    resized_patch = resize_image(patch, output_size)

                    # Save the resized patch to the corresponding folder
                    resized_patch_path = os.path.join(resized_grade_folder, f'resized_{filename}')
                    cv2.imwrite(resized_patch_path, resized_patch)
                    print(f'Resized and saved: {resized_patch_path}')
                else:
                    print(f'Error reading image: {patch_path}')

# Resize the patches in both the training and testing datasets
resize_patches_in_folder(patches_train_folder, resized_train_folder)
resize_patches_in_folder(patches_test_folder, resized_test_folder)

# Print a message indicating the process has completed
print('All images have been resampled to 512x512 and saved to the resized folders.')


TRANSFER LEARNING

In [5]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [6]:
# DEFINE DATA FOLDERS AND CONFIGURE IMAGE DATA GENERATORS

# Define the paths to the training and testing image directories
# `train_dir` contains the resized training images.
# `test_dir` contains the resized testing images.
train_dir = '/dataset tfg final/resizedtrainpatches'
test_dir = '/dataset tfg final/resizedtestpatches'

# Configure the ImageDataGenerator for the training data
# ImageDataGenerator is used to load and preprocess images from the directories, including data augmentation for training.
train_datagen = ImageDataGenerator(
    rotation_range=30,  # Randomly rotate images by up to 30 degrees
    width_shift_range=0.2,  # Randomly shift images horizontally by up to 20% of the image width
    height_shift_range=0.2,  # Randomly shift images vertically by up to 20% of the image height
    zoom_range=0.2,  # Randomly zoom in on images by up to 20%
    horizontal_flip=True,  # Randomly flip images horizontally
    vertical_flip=False,  # Do not flip images vertically
    fill_mode='nearest',  # Fill in any missing pixels after transformation with the nearest pixel
    preprocessing_function=preprocess_input  # Apply DenseNet's preprocessing to normalize the input data
)

# Configure the ImageDataGenerator for the testing data
# For testing, we only rescale the images, and no augmentation is applied.
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input  # Apply DenseNet's preprocessing to normalize the input data
)

# Load the training images using the `train_datagen` generator
# `train_generator` provides batches of augmented training images from the `train_dir`.
train_generator = train_datagen.flow_from_directory(
    train_dir,  # Path to the training images directory
    target_size=(512, 512),  # Resize all images to 512x512 pixels (required by DenseNet)
    batch_size=32,  # Number of images to process in each batch
    class_mode='categorical',  # Use categorical labels (one-hot encoding for multi-class classification)
    shuffle=True  # Shuffle the images during training to ensure randomness
)

# Load the testing images using the `test_datagen` generator
# `test_generator` provides batches of images from the `test_dir` for evaluation.
test_generator = test_datagen.flow_from_directory(
    test_dir,  # Path to the testing images directory
    target_size=(512, 512),  # Resize all images to 512x512 pixels (required by DenseNet)
    batch_size=32,  # Number of images to process in each batch
    class_mode='categorical',  # Use categorical labels for evaluation (one-hot encoding)
    shuffle=False  # Do not shuffle the images during testing (we want to evaluate the images in their original order)
)


Found 114 images belonging to 4 classes.
Found 147 images belonging to 4 classes.


**DENSENET121**

In [ ]:
from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input
from tensorflow.keras.applications import DenseNet121

In [ ]:
# BUILDING THE MODEL WITH DENSENET121 AS THE BASE

# Load the pre-trained DenseNet121 model, excluding the top fully connected layer (include_top=False)
# The 'weights' argument specifies that the model should be initialized with ImageNet weights.
# The 'input_shape' specifies the shape of the input images (512x512 pixels with 3 color channels).
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(512, 512, 3))

# The output of the base model will be passed to additional layers for custom classification
x = base_model.output

# Add a Global Average Pooling layer to reduce the spatial dimensions of the feature map
# This helps reduce the number of parameters and the risk of overfitting.
x = GlobalAveragePooling2D()(x)

# Add a fully connected (dense) layer with 1024 units and ReLU activation
# This layer helps the model learn complex relationships and make predictions.
x = Dense(1024, activation='relu')(x)

# Add the final output layer with the number of classes in the training set (num_classes),
# using a softmax activation function to output probabilities for each class.
# The output layer will have one unit per class in the classification task.
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# Create the model by specifying the input and output layers
# The model will use DenseNet121 as the feature extractor and the newly added layers for classification.
model = Model(inputs=base_model.input, outputs=predictions)


*Without fine-tunning DenseNet121*

In [ ]:
# Loop through all layers in the base model (DenseNet121) and set them as non-trainable
# This is done to keep the pre-trained weights of the base model frozen and avoid modifying them during training.
for layer in base_model.layers:
    layer.trainable = False  # Set each layer in the base model to not be trainable


In [ ]:
# COMPILE THE MODEL WITH SPECIFIED OPTIMIZER, LOSS FUNCTION, AND METRICS

# Compile the model by specifying the optimizer, loss function, and evaluation metrics
# The optimizer is responsible for adjusting the model's weights based on the loss function during training.
# The Adam optimizer is a popular adaptive learning rate method for deep learning models.

model.compile(
    optimizer=Adam(learning_rate=0.001),  # Adam optimizer with a learning rate of 0.001
    loss='categorical_crossentropy',  # Loss function for multi-class classification (used when classes are mutually exclusive)
    metrics=['accuracy']  # Track accuracy during training as the performance metric
)


In [ ]:
# TRAINING THE MODEL WITHOUT FINE-TUNING DENSENET121

# Train the model using the training data generator and validate using the test data generator
# The `fit()` function trains the model on the provided dataset for a specified number of epochs.
# In this case, the model is trained without fine-tuning the DenseNet121 base model, meaning the pre-trained layers are frozen.
history = model.fit(
    train_generator,  # The generator that provides the training data (images and labels)
    epochs=15,  # Number of epochs (iterations over the entire dataset)
    validation_data=test_generator,  # The validation data to evaluate the model after each epoch
)


In [ ]:
# The file format used is HDF5 (.h5), which is commonly used for saving Keras models.
model.save(f'E15model_sin_finetunningdensenet121.h5')  # Save the model
with open('E15history_sin_finetunningdensenet121.json', 'w') as f:
    json.dump(history.history, f)

*With fine-tunning DenseNet121*

In [ ]:
# TRAINING THE MODEL WITH FINE-TUNING DENSENET121

# Freeze the first 108 layers of the base model (DenseNet121) to keep their pre-trained weights
# By freezing the first layers, we allow the model to retain the learned features from ImageNet without updating these weights.
for layer in base_model.layers[:108]:  # Freeze the first 108 layers
    layer.trainable = False

# Unfreeze the layers from 108 onwards, allowing them to be trained
# These layers will be fine-tuned to adjust to the specific task (classification of osteoarthritis grades).
for layer in base_model.layers[108:]:  # Unfreeze layers starting from layer 109
    layer.trainable = True

# Compile the model again after unfreezing layers
# Adam optimizer with a lower learning rate to fine-tune the model without drastically changing the pre-trained features
model.compile(optimizer=Adam(learning_rate=0.0001),  # Adam optimizer with a smaller learning rate for fine-tuning
              loss='categorical_crossentropy',  # Loss function for multi-class classification
              metrics=['accuracy'])  # Track accuracy during training

# Train the model with fine-tuning
# `history_fine` stores the training history, including loss and accuracy metrics during training.
history_fine = model.fit(
    train_generator,  # The generator that provides the training data (images and labels)
    epochs=15,  # Number of epochs (iterations over the entire dataset)
    validation_data=test_generator,  # The validation data to evaluate the model after each epoch
)


In [ ]:
# The 'model.save()' function is used to save the entire model, including its architecture, weights, and training configuration, to a file.
# The file format used is HDF5 (.h5), which is commonly used for saving Keras models.
model.save(f'E15model_con_finetunningdensenet121.h5')  # Save the model with the name 'model1.h5'
with open('E15history_con_finetunningdensenet121.json', 'w') as f:
    json.dump(history_fine.history, f)

**RESNET50**

In [16]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input


In [9]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(512, 512, 3)) #The model was tried without transfer learning, but no results were obtained (weights=None)

In [10]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [11]:
for layer in base_model.layers:
    layer.trainable = False

In [12]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

*WITHOUT FINE TUNNING RESNET50*

In [13]:
# Without fine tunning ResNet50
history = model.fit(
    train_generator,
    epochs=15,
    validation_data=test_generator,

)

Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/4 ━━━━━━━━━━━━━━━━━━━━ 273s 75s/step - accuracy: 0.1739 - loss: 4.0178 - val_accuracy: 0.5374 - val_loss: 2.8856
Epoch 2/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 262s 74s/step - accuracy: 0.2531 - loss: 3.5925 - val_accuracy: 0.3741 - val_loss: 2.9097
Epoch 3/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 261s 73s/step - accuracy: 0.5390 - loss: 1.9174 - val_accuracy: 0.4694 - val_loss: 1.2407
Epoch 4/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 264s 74s/step - accuracy: 0.7186 - loss: 0.6457 - val_accuracy: 0.5578 - val_loss: 1.0972
Epoch 5/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 257s 73s/step - accuracy: 0.6514 - loss: 0.9012 - val_accuracy: 0.5578 - val_loss: 1.4715
Epoch 6/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 261s 77s/step - accuracy: 0.8243 - loss: 0.4599 - val_accuracy: 0.4966 - val_loss: 1.4049
Epoch 7/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 259s 74s/step - accuracy: 0.8486 - loss: 0.4701 - val_accuracy: 0.6667 - val_loss: 1.0978
Epoch 8/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 267s 79s/step - accuracy: 0.8746 - loss: 0.3561 - val_accuracy: 0.6122 - val_loss: 1.3869
Epo

In [15]:
# The 'model.save()' function is used to save the entire model, including its architecture, weights, and training configuration, to a file.
# The file format used is HDF5 (.h5), which is commonly used for saving Keras models.
model.save(f'A15model_sin_finetunningResNet50.h5')  # Save the model with the name 'model1.h5'
with open('A15history_sin_finetunningResNet50.json', 'w') as f:
    json.dump(history.history, f)

*WITH FINE TUNNING RESNET50*

In [ ]:
# TRAINING THE MODEL WITH FINE-TUNING RESNET50

# Freeze the first 45 layers of the base model (ResNet50) to keep their pre-trained weights
# By freezing the first layers, we retain the learned features from ImageNet without modifying these weights.
for layer in base_model.layers[:45]:  # Freeze the first 45 layers
    layer.trainable = False

# Unfreeze the layers from 45 onwards, allowing them to be trained
# These layers will be fine-tuned to adapt to the specific classification task (osteoarthritis grading).
for layer in base_model.layers[45:]:  # Unfreeze layers starting from layer 46
    layer.trainable = True

# Compile the model again after unfreezing layers
# Adam optimizer with a lower learning rate to fine-tune the model without drastically changing the pre-trained features
model.compile(optimizer=Adam(learning_rate=0.0001),  # Adam optimizer with a small learning rate for fine-tuning
              loss='categorical_crossentropy',  # Loss function for multi-class classification
              metrics=['accuracy'])  # Track accuracy during training

# Train the model with fine-tuning
# `history_fine` stores the training history, including loss and accuracy metrics during training.
history_fine = model.fit(
    train_generator,  # The generator that provides the training data (images and labels)
    epochs=15,  # Number of epochs (iterations over the entire dataset)
    validation_data=test_generator,  # The validation data to evaluate the model after each epoch
)


Epoch 1/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 545s 131s/step - accuracy: 0.5504 - loss: 2.3481 - val_accuracy: 0.1633 - val_loss: 7.4621
Epoch 2/15
2/4 ━━━━━━━━━━━━━━━━━━━━ 3:12 96s/step - accuracy: 0.8289 - loss: 0.3623

In [ ]:
# The 'model.save()' function is used to save the entire model, including its architecture, weights, and training configuration, to a file.
# The file format used is HDF5 (.h5), which is commonly used for saving Keras models.
model.save(f'M15modelcon_finetunningResNet50.h5')  # Save the model with the name 'model1.h5'
with open('M15history_con_finetunningResNet50.json', 'w') as f:
    json.dump(history_fine.history, f)

**EFFICIENTNETV2B0**

In [1]:
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input

In [9]:
# Load the pre-trained EfficientNetV2B0 model, excluding the top fully connected layer (include_top=False)
# The 'weights' argument specifies that the model should be initialized with ImageNet weights.
# The 'input_shape' specifies the shape of the input images (512x512 pixels with 3 color channels).
base_model = EfficientNetV2B0(weights='imagenet', include_top=False, input_shape=(512, 512, 3))  #The model was tried without transfer learning, but no results were obtained (weights=None)

# The output of the base model will be passed to additional layers for custom classification
x = base_model.output

# Add a Global Average Pooling layer to reduce the spatial dimensions of the feature map
# This helps reduce the number of parameters and prevent overfitting.
x = GlobalAveragePooling2D()(x)

# Add a fully connected (dense) layer with 1024 units and ReLU activation
# This layer helps the model learn complex relationships from the features extracted by the base model.
x = Dense(1024, activation='relu')(x)

# Add the final output layer with the number of classes in the training set (num_classes),
# using a softmax activation function to output probabilities for each class.
# This layer will predict the class of each image based on the features learned.
predictions = Dense(train_generator.num_classes, activation='softmax')(x)  # Number of classes taken from the train folder

# Create the model by specifying the input and output layers
# The model uses EfficientNetV2B0 as the feature extractor and the added layers for classification.
model = Model(inputs=base_model.input, outputs=predictions)

In [10]:
for layer in base_model.layers:
    layer.trainable = False

In [11]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

*WITHOUT FINE TUNNING EFFICIENTNETV2B0*

In [12]:
#without fine tunning
history = model.fit(
    train_generator,
    epochs=15,
    validation_data=test_generator,

)

Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/4 ━━━━━━━━━━━━━━━━━━━━ 132s 32s/step - accuracy: 0.4562 - loss: 1.2164 - val_accuracy: 0.6259 - val_loss: 0.7387
Epoch 2/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 155s 37s/step - accuracy: 0.7423 - loss: 0.6070 - val_accuracy: 0.4422 - val_loss: 1.5362
Epoch 3/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 111s 28s/step - accuracy: 0.7482 - loss: 0.6546 - val_accuracy: 0.6871 - val_loss: 0.7094
Epoch 4/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 142s 26s/step - accuracy: 0.7653 - loss: 0.4892 - val_accuracy: 0.6599 - val_loss: 0.7490
Epoch 5/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 130s 37s/step - accuracy: 0.9172 - loss: 0.2487 - val_accuracy: 0.6395 - val_loss: 1.0290
Epoch 6/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 115s 32s/step - accuracy: 0.9323 - loss: 0.2552 - val_accuracy: 0.7211 - val_loss: 0.7339
Epoch 7/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 125s 28s/step - accuracy: 0.9307 - loss: 0.1697 - val_accuracy: 0.7143 - val_loss: 0.7665
Epoch 8/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 130s 37s/step - accuracy: 0.9198 - loss: 0.2171 - val_accuracy: 0.5646 - val_loss: 1.1988
Epo

In [13]:
# The 'model.save()' function is used to save the entire model, including its architecture, weights, and training configuration, to a file.
# The file format used is HDF5 (.h5), which is commonly used for saving Keras models.
model.save(f'M15model_sin_finetunningrEfficientnetv2b0.h5')  # Save the model with the name 'model1.h5'
with open('M15history_sin_finetunningrEfficientnetv2b0.json', 'w') as f:
    json.dump(history.history, f)

*WITH FINE TUNNING EFFICIENTNETV2B0*

In [ ]:
# FINE-TUNING THE EFFICIENTNETV2B0 MODEL

# Freeze the first 245 layers of the base model (EfficientNetV2B0) to retain their pre-trained weights
# By freezing the first layers, we ensure that the low-level features (such as edges, textures, etc.)
# learned from ImageNet are not modified during training.
for layer in base_model.layers[:245]:  # Freeze the first 245 layers
    layer.trainable = False

# Unfreeze the layers from 245 onwards, allowing them to be trained
# These layers will be fine-tuned to adjust to the specific classification task (such as the osteoarthritis grading).
for layer in base_model.layers[245:]:  # Unfreeze layers from layer 246 onwards
    layer.trainable = True

# Compile the model again after unfreezing layers
# Using the Adam optimizer with a smaller learning rate (0.0001) to fine-tune the model.
# This allows the model to make small adjustments to the pre-trained features while learning new features for the task.
model.compile(optimizer=Adam(learning_rate=0.0001),  # Adam optimizer with a small learning rate for fine-tuning
              loss='categorical_crossentropy',  # Loss function for multi-class classification
              metrics=['accuracy'])  # Track accuracy during training

# Train the model with fine-tuning
# `history_fine` stores the training history, including the loss and accuracy metrics during training.
history_fine = model.fit(
    train_generator,  # The generator that provides the training data (images and labels)
    epochs=15,  # Number of epochs (iterations over the entire dataset)
    validation_data=test_generator,  # The validation data to evaluate the model after each epoch
)


Epoch 1/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 133s 30s/step - accuracy: 0.8341 - loss: 0.5044 - val_accuracy: 0.5986 - val_loss: 1.4626
Epoch 2/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 164s 38s/step - accuracy: 0.9351 - loss: 0.2310 - val_accuracy: 0.6122 - val_loss: 1.3821
Epoch 3/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 107s 28s/step - accuracy: 0.9482 - loss: 0.2212 - val_accuracy: 0.6122 - val_loss: 1.2626
Epoch 4/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 140s 28s/step - accuracy: 0.9585 - loss: 0.1353 - val_accuracy: 0.6327 - val_loss: 1.1664
Epoch 5/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11s/step - accuracy: 0.9700 - loss: 0.1427 

In [ ]:
# The 'model.save()' function is used to save the entire model, including its architecture, weights, and training configuration, to a file.
# The file format used is HDF5 (.h5), which is commonly used for saving Keras models.
model.save(f'E15model_con_finetunningrEfficientnetv2b0.h5')  # Save the model with the name 'model1.h5'
with open('E15history_con_finetunningrEfficientnetv2b0.json', 'w') as f:
    json.dump(history.history, f)